In [ ]:
# LOAD RELEVANT LIBRARIES
import numpy as np
from kilosort import run_kilosort
from pathlib import Path

In [ ]:
# DEFINE DIRECTORIES AND DATA STRUCTURES
filename = 'troy-agl_t-2021-06-25'

# Get the directory of the original SAVE_PATH
bin_dir = Path('C:\\KIKUCHI-LOCAL\\data\\ephys\\ks\\troy-agl_t-2021-06-25')

# Create a new Path object with the filename and .bin extension
bin_file = bin_dir / f"{filename}.dat"


In [ ]:
# DEFINE SESSION INFORMATION
sampling_frequency = 32000
electrode_spacing = 200

# Load probe file based on electrode spacing
session_probe = 'plexon_200um_32ch.prb';

In [ ]:
# Define Kilosort parameters
settings = {'data_dir': bin_dir, 'n_chan_bin': 32, 'fs': sampling_frequency, 'batch_size': 512000, 'nblocks': 1,
            'Th_universal': 9, 'Th_learned': 7, 'tmin': 0, 'tmax': np.inf, 'nt0min': None,
            'dmin': None, 'min_template_size': 10, 'template_sizes': 5, 'nearest_chans': 5,
            'nearest_templates': 1, 'n_templates': 6, 'n_pcs': 6, 'Th_single_ch': 3}


In [ ]:
# RUN KILOSORT
# NOTE: 'n_chan_bin' is a required setting, and should reflect the total number
#       of channels in the binary file. For information on other available
#       settings, see `kilosort.run_kilosort.default_settings`.

run_kilosort(settings=settings, probe_name=session_probe)